<a href="https://colab.research.google.com/github/axel-sirota/neural-networks-image-classification/blob/main/Module2Demo1_CNNClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 40.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
# Step 1: Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
import os
from PIL import Image
import io


In [ ]:
# Step 2: Download and prepare the dataset from Hugging Face
dataset = load_dataset('keremberke/chest-xray-classification', 'full')

# Prepare directories
os.makedirs('xray_dataset/train/normal', exist_ok=True)
os.makedirs('xray_dataset/train/pneumonia', exist_ok=True)
os.makedirs('xray_dataset/validation/normal', exist_ok=True)
os.makedirs('xray_dataset/validation/pneumonia', exist_ok=True)


# # Assuming the dataset splits are already defined in Hugging Face dataset
for split in ['train', 'validation']:
    images = dataset[split]['image']
    labels = dataset[split]['labels']

    for i, (img, label) in enumerate(zip(images, labels)):
        if label == 0:
            label_folder = 'normal'
        else:
            label_folder = 'pneumonia'
        img_path = f'xray_dataset/{split}/{label_folder}/{split}_{label}_{i}.jpeg'
        buffer = io.BytesIO()
        img.save(buffer, format='JPEG')
        with open(img_path, 'wb') as f:
            f.write(buffer.getvalue())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

# Step 3: Preprocess the dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_dir = 'xray_dataset/train'
validation_dir = 'xray_dataset/validation'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)


In [ ]:

# Step 4: Build the CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:


# Step 5: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:

# Step 6: Train the model
history = model.fit(
    train_generator,
    batch_size=32,
    epochs=15,
    validation_data=validation_generator
    )
